In [ ]:
import os
import time
import psutil
import subprocess
import logging
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler

# 로깅 설정
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')

# 스크립트 중복 실행 방지
def is_script_running(script_name):
    for proc in psutil.process_iter(['name', 'cmdline']):
        cmdline = proc.info.get('cmdline')
        if cmdline and script_name in ' '.join(cmdline):
            return True
    return False

# 파일 준비 상태 확인
def is_file_ready(filepath):
    retry_count = 0
    max_retries = 10  # 최대 재시도 횟수
    while retry_count < max_retries:
        try:
            with open(filepath, 'rb'):  # 파일 읽기 시도
                return True
        except IOError:
            retry_count += 1
            logging.info(f"파일이 잠겨 있습니다. 대기 중({retry_count}/{max_retries}): {filepath}")
            time.sleep(1)
    logging.error(f"파일 준비 실패: {filepath}")
    return False

processed_files = set()  # 중복 실행 방지

class Watcher:
    def __init__(self, path):
        self.path = path
        self.observer = Observer()

    def run(self):
        if is_script_running("FINAL_CODE_S1.py") or is_script_running("FINAL_CODE_S2.py"):
            logging.info("스크립트가 이미 실행 중입니다.")
            return

        event_handler = Handler()
        self.observer.schedule(event_handler, self.path, recursive=False)
        self.observer.start()
        logging.info(f"Watching for changes in: {self.path}")

        try:
            while True:
                time.sleep(1)
        except KeyboardInterrupt:
            self.observer.stop()
            logging.info("Observer Stopped")
        except Exception as e:
            logging.error(f"Unexpected error in Watcher: {e}")
        self.observer.join()


class Handler(FileSystemEventHandler):
    def on_created(self, event):
        if event.is_directory or event.src_path in processed_files:
            return None

        file_name = os.path.basename(event.src_path)
        logging.info(f"Detected new file: {file_name}")

        if file_name.endswith('_Quant_data_original.xlsx'):
            if not is_file_ready(event.src_path):
                logging.error(f"파일 처리 실패: {event.src_path}")
                return

            processed_files.add(event.src_path)  # 처리된 파일로 등록
            logging.info(f"새 파일 처리 시작: {file_name}")

            try:
                logging.info("Running FINAL_CODE_S1.py...")
                result = subprocess.run(["python", "FINAL_CODE_S1.py", event.src_path], capture_output=True, text=True, encoding='utf-8')
                logging.info(f"FINAL_CODE_S1.py stdout: {result.stdout}")
                logging.error(f"FINAL_CODE_S1.py stderr: {result.stderr}")
                result.check_returncode()

                logging.info("Running FINAL_CODE_S2.py...")
                result = subprocess.run(["python", "FINAL_CODE_S2.py", event.src_path], capture_output=True, text=True, encoding='utf-8')
                logging.info(f"FINAL_CODE_S2.py stdout: {result.stdout}")
                logging.error(f"FINAL_CODE_S2.py stderr: {result.stderr}")
                result.check_returncode()

            except subprocess.CalledProcessError as e:
                logging.error(f"Error while executing subprocess: {e}")
            except Exception as e:
                logging.error(f"Unexpected error in Handler: {e}")


if __name__ == "__main__":
    path = r"Z:\남수경\quant\watchdog_test"

    if not os.path.exists(path):
        logging.error(f"경로에 접근할 수 없습니다: {path}")
        exit(1)

    if is_script_running("FINAL_CODE_S1.py") or is_script_running("FINAL_CODE_S2.py"):
        logging.info("스크립트가 이미 실행 중입니다. 프로그램 종료.")
        exit(1)

    w = Watcher(path)
    w.run()


2024-12-09 16:48:09,341 - INFO - Watching for changes in: Z:\남수경\quant\watchdog_test
2024-12-09 16:48:24,874 - INFO - Detected new file: 20240917__Quant_data_original.xlsx
2024-12-09 16:48:24,874 - INFO - 파일이 잠겨 있습니다. 대기 중(1/10): Z:\남수경\quant\watchdog_test\20240917__Quant_data_original.xlsx
2024-12-09 16:48:25,925 - INFO - 새 파일 처리 시작: 20240917__Quant_data_original.xlsx
2024-12-09 16:48:25,925 - INFO - Running FINAL_CODE_S1.py...
2024-12-09 16:48:27,793 - INFO - FINAL_CODE_S1.py stdout: 
2024-12-09 16:48:27,793 - ERROR - FINAL_CODE_S1.py stderr: Traceback (most recent call last):
  File "z:\남수경\quant\watchdog_test\FINAL_CODE_S1.py", line 11, in <module>
    df = pd.read_excel(file_path, sheet_name='1차', skiprows=12)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pandas\io\excel\_base.py", line 495, in read_excel
    io = Exce